In [16]:
import tensorflow as tf
import os

In [17]:
# Variables
W = tf.Variable(tf.zeros([4, 3]), name="weights")
b = tf.Variable(tf.zeros([3]), name="bias")

In [18]:
def combine_input(X):
    return tf.matmul(X, W) + b

In [19]:
def inference(X):
    return tf.nn.softmax(combine_input(X))

In [20]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=combine_input(X)))

In [32]:
def read_csv(batch_size, file_name, record_defaults):
    # filename_queue = tf.train.string_input_producer([os.path.dirname(__file__) + "/datasets/Iris/" + file_name])
    filename_queue = tf.train.string_input_producer([os.getcwd() + "/datasets/Iris/" + file_name])

        
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    # 数据文件中不得有空格，导致错误 Expect 5 fields but have 0 in record 0
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    
    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size*50, min_after_dequeue=batch_size)

In [33]:
def inputs():
    sepal_length, sepal_width, petal_length, petal_width, label = \
        read_csv(100, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iras-setosa"]),
        tf.equal(label, ["Iras-versicolor"]),
        tf.equal(label, ["Iras-verginica"])
    ])), 0))
    # 为什么加上0
    
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))
    return features, label_number

In [34]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [35]:
def evaluate(sess, X, Y):
    predicted = tf.cast(tf.arg_max(inference(X),1), tf.int32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [37]:
# Session
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 实际训练迭代次数
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # 出于调试和学习的目的，查看损失在训练过程中递减的情况
        if step % 10 == 0:
            print("loss: ", sess.run([total_loss]))
    
    evaluate(sess, X, Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [0.7543695]
loss:  [0.12879914]
loss:  [0.077960745]
loss:  [0.039993946]
loss:  [0.035650834]
loss:  [0.033608414]
loss:  [0.023581915]
loss:  [0.019823916]
loss:  [0.01680941]
loss:  [0.018071966]
loss:  [0.015662894]
loss:  [0.012717009]
loss:  [0.012551411]
loss:  [0.012821569]
loss:  [0.010764234]
loss:  [0.011153067]
loss:  [0.0075856578]
loss:  [0.008880767]
loss:  [0.008010131]
loss:  [0.008034318]
loss:  [0.009094037]
loss:  [0.0065321154]
loss:  [0.007109667]
loss:  [0.0075289183]
loss:  [0.0059227855]
loss:  [0.0069184713]
loss:  [0.0061617075]
loss:  [0.005739308]
loss:  [0.005588323]
loss:  [0.0045098583]
loss:  [0.0049349717]
loss:  [0.0054517672]
loss:  [0.0046364926]
loss:  [0.0053009056]
loss:  [0.003927271]
loss:  [0.004378805]
loss:  [0.004901467]
loss:  [0.0039172997]
loss:  [0.003965969]
loss:  [0.0047602197]
loss:  [0.003693986]
loss:  [0.004093641]
loss:  [0.0032287147]
loss:  [0.0040862365]
loss:  [0.0040523806]
loss:  [0.003256374]
loss:  [0.0033431007]
